In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import math

In [2]:
df = pd.read_csv("dataset_output/libras_ufop/libras_ufop_openpose_60_frames.csv")

In [8]:
df = df.drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1)

In [3]:
person_videos = 15
persons = 5
frames = 60
person = []
for i in range(persons):
    person.extend([i] * frames * person_videos)
person = person * len(df["category"].unique())

In [20]:
person = df["video_name"].str[-8:-6]

In [21]:
df["person"] = person

In [4]:
df

,category,video_name,frame,hand_0_0_x,hand_0_0_y,hand_0_0_z,hand_0_1_x,hand_0_1_y,hand_0_1_z,hand_0_2_x,...,pose_22_z,pose_23_x,pose_23_y,pose_23_z,pose_24_x,pose_24_y,pose_24_z,missing_hand,missing_face,person
0,1,p1_c1_s1_0,1,0.555086,0.561309,0.396433,0.547985,0.570326,0.402954,0.542236,...,0.751818,0.433051,0.966384,0.792620,0.449503,0.930953,0.673173,False,False,1.0
1,1,p1_c1_s1_0,2,0.552441,0.554763,0.376937,0.544950,0.563844,0.328017,0.539501,...,0.721404,0.431155,0.966412,0.771990,0.449501,0.928337,0.667928,False,False,1.0
2,1,p1_c1_s1_0,3,0.540484,0.558278,0.014662,0.539830,0.565252,0.043459,0.532311,...,0.745815,0.433046,0.966367,0.777363,0.449523,0.928312,0.661847,False,False,1.0
3,1,p1_c1_s1_0,4,0.539812,0.537244,0.100900,0.534094,0.548904,0.083215,0.531067,...,0.697552,0.433064,0.966390,0.749232,0.449475,0.930948,0.649300,False,False,1.0
4,1,p1_c1_s1_0,5,0.530725,0.526807,0.142478,0.524336,0.534430,0.150466,0.517610,...,0.727615,0.433072,0.966384,0.772875,0.449477,0.928342,0.647910,False,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182395,47,p5_c4_s9_10,55,0.522440,0.436927,0.275950,0.533989,0.446403,0.045277,0.532805,...,0.705954,0.426990,0.950059,0.665721,0.437205,0.898362,0.538676,False,False,5.0
182396,47,p5_c4_s9_10,56,0.522440,0.436927,0.275950,0.533989,0.446403,0.045277,0.532805,...,0.705954,0.426990,0.950059,0.665721,0.437205,0.898362,0.538676,False,False,5.0
182397,47,p5_c4_s9_10,57,0.522440,0.436927,0.275950,0.533989,0.446403,0.045277,0.532805,...,0.705954,0.426990,0.950059,0.665721,0.437205,0.898362,0.538676,False,False,5.0
182398,47,p5_c4_s9_10,58,0.522440,0.436927,0.275950,0.533989,0.446403,0.045277,0.532805,...,0.705954,0.426990,0.950059,0.665721,0.437205,0.898362,0.538676,False,False,5.0


In [5]:
df["category"].unique()

array([ 1, 10,  2,  3,  4,  5,  6,  7,  8,  9, 11, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 12, 30, 13, 14, 15, 16, 17, 18, 19, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 40, 41, 42,
       43, 44, 45, 46, 47])

In [6]:
landmarks_name = np.array([i for i in list(df.columns) if i not in ["Unnamed: 0", "Unnamed: 0.1", "Unnamed: 0.2", "category", "category_index", "video_name", "frame", "missing_hand", "missing_face", "person"]])
landmarks_name = landmarks_name.reshape((int(landmarks_name.shape[0]/3), 3))

In [7]:
# GPT Code
def rotate_landmarks(landmarks, rotation_angle):
    # Assuming landmarks are in (x, y, z) format
    
    # Calculate the centroid (center) of the landmarks
#     centroid = np.mean(landmarks, axis=0)
    centroid = 0.5

    # Translate landmarks to the origin (center)
    translated_landmarks = landmarks - centroid
    # Create a rotation matrix for the given angle (in radians)
    rotation_matrix = np.array([[np.cos(rotation_angle), -np.sin(rotation_angle), 0],
                                 [np.sin(rotation_angle), np.cos(rotation_angle), 0],
                                 [0, 0, 1]])
    
    # Apply the rotation to each landmark
#     print(rotation_matrix)
    rotated_landmarks = np.dot(translated_landmarks, rotation_matrix)
    
    return rotated_landmarks + centroid

In [8]:
def zoom_landmarks(landmarks, zoom_factor):
    # Scale the landmarks by the zoom factor
    zoomed_landmarks = landmarks * zoom_factor
    
    return zoomed_landmarks

In [9]:
def translate_landmarks(landmarks, translation_vector):
    # Translate the landmarks by the given vector
    translated_landmarks = landmarks + translation_vector
    
    return translated_landmarks

In [10]:
def apply_transformation(frame, rotation, zoom, translate_x, translate_y):
    for lx, ly, lz in landmarks_name:
        result = rotate_landmarks(frame[[lx, ly, lz]], math.radians(rotation))
        frame[[lx, ly, lz]] = result
        result = zoom_landmarks(frame[[lx, ly, lz]], zoom)
        frame[[lx, ly, lz]] = result
        result = translate_landmarks(frame[[lx, ly, lz]], [translate_x, translate_x, 0])
        frame[[lx, ly, lz]] = result
    return frame

In [11]:
np.random.normal(0, 0.5, 10)

array([ 0.22999072, -0.51184415, -0.50377719, -0.09211836,  0.11794797,
       -0.0042867 ,  0.57190318,  0.49895801, -0.33572188, -0.24208063])

In [12]:
sample_size = 0.4
samples_creation = 4
rotation_sigma = 4
zoom_sigma = 0.05
translate_x_sigma = 0.01
translate_y_sigma = 0.0

In [13]:
df_augmented = pd.DataFrame(columns=df.columns)
for category in df["category"].unique():
    print("Processing category", category)
    df_category = df[df["category"] == category]
    videos = df_category["video_name"].unique()
    videos = np.random.choice(videos, int(len(videos) * sample_size))
    for i in range(samples_creation):
        for video in videos:
            df_video = df[df["video_name"] == video].copy()
            rotation = np.random.normal(0, rotation_sigma)
            zoom = np.random.normal(0, zoom_sigma) + 1
            translate_x = np.random.normal(0, translate_x_sigma)
            translate_y = np.random.normal(0, translate_y_sigma)
            df_video = apply_transformation(df_video, rotation, zoom, translate_x, translate_y)
#             for index, frame in df_video.iterrows():
#                 frame = apply_transformation(frame, rotation, zoom, translate_x, translate_y)
#                 df_video[index] = frame
            df_video["video_name"] = f"augmented{i}-" + df_video["video_name"]
            df_augmented = pd.concat([df_augmented, df_video])

Processing category 1
Processing category 10
Processing category 2
Processing category 3
Processing category 4
Processing category 5
Processing category 6
Processing category 7
Processing category 8
Processing category 9
Processing category 11
Processing category 21
Processing category 22
Processing category 23
Processing category 24
Processing category 25
Processing category 26
Processing category 27
Processing category 28
Processing category 29
Processing category 12
Processing category 30
Processing category 13
Processing category 14
Processing category 15
Processing category 16
Processing category 17
Processing category 18
Processing category 19
Processing category 31
Processing category 32
Processing category 33
Processing category 34
Processing category 35
Processing category 36
Processing category 37
Processing category 38
Processing category 39
Processing category 48
Processing category 49
Processing category 50
Processing category 51
Processing category 52
Processing category 

In [14]:
df_augmented

,category,video_name,frame,hand_0_0_x,hand_0_0_y,hand_0_0_z,hand_0_1_x,hand_0_1_y,hand_0_1_z,hand_0_2_x,...,pose_22_z,pose_23_x,pose_23_y,pose_23_z,pose_24_x,pose_24_y,pose_24_z,missing_hand,missing_face,person
74340,1,augmented0-p3_c1_s1_1,1,0.567052,0.561045,0.401563,0.562729,0.572573,0.539928,0.559859,...,0.760518,0.496127,0.981239,0.728293,0.529134,0.951552,0.762617,False,False,3.0
74341,1,augmented0-p3_c1_s1_1,2,0.567373,0.560770,0.414415,0.562416,0.572402,0.569638,0.558904,...,0.761729,0.494572,0.984233,0.718332,0.529105,0.951512,0.754282,False,False,3.0
74342,1,augmented0-p3_c1_s1_1,3,0.567081,0.561039,0.405918,0.562479,0.573085,0.519100,0.557523,...,0.770804,0.496533,0.983909,0.742758,0.528670,0.948890,0.736955,False,False,3.0
74343,1,augmented0-p3_c1_s1_1,4,0.566396,0.559402,0.453946,0.561872,0.571469,0.544566,0.556685,...,0.758050,0.494534,0.984223,0.710625,0.528736,0.948862,0.774872,False,False,3.0
74344,1,augmented0-p3_c1_s1_1,5,0.565975,0.559273,0.387985,0.561202,0.570620,0.489898,0.558281,...,0.733842,0.495777,0.978582,0.731643,0.529026,0.951513,0.708776,False,False,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146335,47,augmented3-p4_c4_s9_9,55,0.558719,0.543551,0.359298,0.554663,0.550149,0.384023,0.551655,...,0.722236,0.481820,0.903522,0.662009,0.485298,0.867864,0.600545,False,False,4.0
146336,47,augmented3-p4_c4_s9_9,56,0.557869,0.543550,0.224280,0.557763,0.555297,0.337448,0.554575,...,0.731985,0.479992,0.903571,0.660488,0.483457,0.867890,0.600386,False,False,4.0
146337,47,augmented3-p4_c4_s9_9,57,0.560119,0.565282,0.211288,0.556116,0.569323,0.395911,0.554807,...,0.734038,0.481797,0.903435,0.660955,0.485275,0.867904,0.609904,False,False,4.0
146338,47,augmented3-p4_c4_s9_9,58,0.559930,0.558377,0.262745,0.557399,0.569536,0.408430,0.556634,...,0.720383,0.479968,0.903578,0.652971,0.485295,0.867777,0.600907,False,False,4.0


In [ ]:
pd.concat([df, df_augmented]).to_csv("dataset_output/libras_ufop/libras_ufop_openpose_60_frames_augmented.csv")

In [126]:
# df_augmented.to_csv("datasets/lsa64_augmentation_test.csv")